## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.
c:\users\henry\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\henry\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\henry\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
LEARNING_RATE = 1e-3
EPOCHS = 40 # IF you feel too run to finish, try to make it smaller
BATCH_SIZE = 1024 

In [9]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
reduce_lr_factor = [0.3,0.5,0.8]
redice_lr_patient = [5,10,20]

In [ ]:
import itertools
results = {}
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i" % (i, reduce_factor, reduce_patient))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    
    """Code Here
    設定 reduce learning rate 的 callback function
    """
    reduce_lr = ReduceLROnPlateau(factor=reduce_factor, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=reduce_patient, 
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, reduce_factor: 0.30, reduce_patient: 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalizatio

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 7s 141us/step - loss: 3.0072 - acc: 0.1117 - val_loss: 2.8495 - val_acc: 0.1511
Epoch 2/40
50000/50000 [==============================] - 6s 112us/step - loss: 2.6556 - acc: 0.1666 - val_loss: 2.6346 - val_acc: 0.1883
Epoch 3/40
50000/50000 [==============================] - 6s 126us/step - loss: 2.4580 - acc: 0.2054 - val_loss: 2.4570 - val_acc: 0.2172
Epoch 4/40
50000/50000 [==============================] - 6s 129us/step - loss: 2.3417 - acc: 0.2307 - val_loss: 2.3472 - val_acc: 0.2375
Epoch 5/40
50000/50000 [==============================] - 6s 127us/step - loss: 2.2617 - acc: 0.2483 - val_loss: 2.2763 - val_acc: 0.2530
Epoch 6/40
50000/50000 [==============================] - 6s 118us/step - loss: 2.2040 - acc: 0.2634 - val_loss: 2.2202 - val_acc: 0.2640
Epoch 7/40
50000/50000 [==============================] - 6s 111us/step - loss: 2.1552 - acc: 0.2744 - val_loss: 2.1769 - 

Epoch 9/40
50000/50000 [==============================] - 6s 115us/step - loss: 2.0715 - acc: 0.2957 - val_loss: 2.0940 - val_acc: 0.2918
Epoch 10/40
50000/50000 [==============================] - 6s 117us/step - loss: 2.0428 - acc: 0.3036 - val_loss: 2.0711 - val_acc: 0.2989
Epoch 11/40
50000/50000 [==============================] - 6s 111us/step - loss: 2.0182 - acc: 0.3109 - val_loss: 2.0505 - val_acc: 0.3046
Epoch 12/40
50000/50000 [==============================] - 6s 111us/step - loss: 1.9973 - acc: 0.3162 - val_loss: 2.0312 - val_acc: 0.3101
Epoch 13/40
50000/50000 [==============================] - 6s 115us/step - loss: 1.9775 - acc: 0.3209 - val_loss: 2.0140 - val_acc: 0.3131
Epoch 14/40
50000/50000 [==============================] - 6s 112us/step - loss: 1.9594 - acc: 0.3266 - val_loss: 2.0001 - val_acc: 0.3191
Epoch 15/40
50000/50000 [==============================] - 5s 109us/step - loss: 1.9422 - acc: 0.3317 - val_loss: 1.9859 - val_acc: 0.3238
Epoch 16/40
50000/50000 [===

50000/50000 [==============================] - 5s 105us/step - loss: 1.9225 - acc: 0.3447 - val_loss: 1.9799 - val_acc: 0.3342
Epoch 18/40
50000/50000 [==============================] - 5s 105us/step - loss: 1.9070 - acc: 0.3515 - val_loss: 1.9652 - val_acc: 0.3388
Epoch 19/40
50000/50000 [==============================] - 5s 105us/step - loss: 1.8924 - acc: 0.3538 - val_loss: 1.9525 - val_acc: 0.3414
Epoch 20/40
50000/50000 [==============================] - 5s 107us/step - loss: 1.8787 - acc: 0.3587 - val_loss: 1.9408 - val_acc: 0.3434
Epoch 21/40
50000/50000 [==============================] - 5s 107us/step - loss: 1.8666 - acc: 0.3611 - val_loss: 1.9295 - val_acc: 0.3470
Epoch 22/40
50000/50000 [==============================] - 5s 106us/step - loss: 1.8544 - acc: 0.3666 - val_loss: 1.9187 - val_acc: 0.3481
Epoch 23/40
50000/50000 [==============================] - 5s 106us/step - loss: 1.8438 - acc: 0.3693 - val_loss: 1.9093 - val_acc: 0.3520
Epoch 24/40
50000/50000 [==============

50000/50000 [==============================] - 5s 107us/step - loss: 1.8067 - acc: 0.3743 - val_loss: 1.8692 - val_acc: 0.3586
Epoch 26/40
50000/50000 [==============================] - 5s 105us/step - loss: 1.7971 - acc: 0.3781 - val_loss: 1.8612 - val_acc: 0.3603
Epoch 27/40
50000/50000 [==============================] - 5s 106us/step - loss: 1.7891 - acc: 0.3804 - val_loss: 1.8546 - val_acc: 0.3629
Epoch 28/40
50000/50000 [==============================] - 5s 105us/step - loss: 1.7804 - acc: 0.3833 - val_loss: 1.8481 - val_acc: 0.3651
Epoch 29/40
50000/50000 [==============================] - 5s 105us/step - loss: 1.7726 - acc: 0.3852 - val_loss: 1.8418 - val_acc: 0.3650
Epoch 30/40
50000/50000 [==============================] - 5s 106us/step - loss: 1.7643 - acc: 0.3881 - val_loss: 1.8362 - val_acc: 0.3683
Epoch 31/40
50000/50000 [==============================] - 5s 106us/step - loss: 1.7564 - acc: 0.3916 - val_loss: 1.8301 - val_acc: 0.3704
Epoch 32/40
50000/50000 [==============

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()